# Computational Proteomics Trends 2019. 

This jupyter notebook aims to read the data from Google Scholar csv poll into a panda dataframe to perform basic statistics about proteomics software useage by 2019. 


In [39]:
import pandas as pd 
#import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.plotly as py

init_notebook_mode(connected=True)      

# Read the poll data 

data_df = pd.read_csv("../data/poll-results-07032019.csv")

def get_pie_chart(collection): 
    """
    This function returns a pie chart from a collection of elements. 
    """
    collection_counts = {} 
    for word in collection:
        token_collection = str(word).split(";")
        for token in token_collection: 
            if token.strip() != 'nan':
                if token not in collection_counts:
                   collection_counts[token] = 0
                collection_counts[token] = collection_counts[token] + 1

  
    trace=go.Pie(labels=[k  for  k in  collection_counts.keys()],values=[k  for  k in  collection_counts.values()])
    return trace

data_df.head() 

,Timestamp,Username,What is your current position?,In what kind of research institute do you work?,"Which of these software do you use for your Quant Analysis. [PeptideAtlas (StPeter, Libra, XPress)]",Which of these software do you use for your Quant Analysis. [OpenMS],Which of these software do you use for your Quant Analysis. [Other],Which of these software do you use for your Quant Analysis. [MSnBase (PMID:22113085)],Which of these software do you use for your Quant Analysis. [moFF],Which of these software do you use for your Quant Analysis. [FlashLFQ (PMID:29083185)],...,Search engine and type of analysis [MSAmanda],Search engine and type of analysis [Mascot],"Which type of Protein quant data you would like to see more in proteomics resources. (e.g. PeptideAtlas, ProteomicsDB, etc) [Label-Free Differential expression experiments]","Which type of Protein quant data you would like to see more in proteomics resources. (e.g. PeptideAtlas, ProteomicsDB, etc) [Label-Free Base line expression experiments]","Which type of Protein quant data you would like to see more in proteomics resources. (e.g. PeptideAtlas, ProteomicsDB, etc) [TMT Differential expression experiments]","Which type of Protein quant data you would like to see more in proteomics resources. (e.g. PeptideAtlas, ProteomicsDB, etc) [TMT Base line expression experiments]","Which type of Protein quant data you would like to see more in proteomics resources. (e.g. PeptideAtlas, ProteomicsDB, etc) [SRM/MR expression Experiments]","Which type of Protein quant data you would like to see more in proteomics resources. (e.g. PeptideAtlas, ProteomicsDB, etc) [DIA/SWATH expression experiments]",In case you would like to see something else more represented in Proteomics resources let us know.,Unnamed: 66
0,2019/02/08 1:42:21 PM GMT,user1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019/02/08 1:49:09 PM GMT,user2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019/02/08 1:54:35 PM GMT,user3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019/02/08 2:13:28 PM GMT,user4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019/02/08 3:22:10 PM GMT,user5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Label Free Analysis MS1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Position Distribution

Read the position and occupation of every participant of the poll. 

In [40]:
positions = data_df.iloc[:,2]
trace = get_pie_chart(positions)
iplot([trace])

## Affiliation distribtuion 

The number of participants in the poll by type of Institution. 

In [41]:
affiliations = data_df.iloc[:,3]
trace = get_pie_chart(affiliations)     
iplot([trace])

In [ ]:
## Which of these software do you use for your Quant Analysis

These plots will show the usage of Quantification software. 

In [67]:
question = "Which of these software do you use for your Quant Analysis"
column_names = [col for col in data_df.columns if question in col]
values = []
for column in column_names: 
    value = column[column.find("[") + 1: column.find("]")]
    values.append(value)
columns = data_df.filter(regex=question)
#print(columns)

plot_values = {}

for index, row in columns.iterrows():
    for column in column_names: 
        column_value = column[column.find("[") + 1: column.find("]")]
        complete_value = columns.at[index,column] 
        combined_values = str(complete_value).split(";")
        for value in combined_values: 
            if value in values: 
                print("Skip this value, is old style")
            elif value in plot_values:
                print(column_value, value)
                if column_value in plot_values[value]:
                    plot_values[value][column_value] = plot_values[value][column_value] +1
                elif ((str(value)).lower() != 'nan'): 
                    plot_values[value] = {}
                    plot_values[value][column_value]=1
            elif((str(value)).lower() != 'nan'):
                plot_values[value] = {}
                plot_values[value][column_value]=1
print(plot_values)
data = [] 
for option in plot_values: 
    print(option)
    print("\n")
    trace = go.Bar(x = plot_values[option].keys(), y = plot_values[option].values(),name = option)
    data.append(trace)   
        
#trace1 = go.Bar(
#    x=['giraffes', 'orangutans', 'monkeys'],
#    y=[20, 14, 23],
#    name='SF Zoo'
#)
#trace2 = go.Bar(
#    x=['giraffes', 'orangutans', 'monkeys'],
#    y=[12, 18, 29],
#    name='LA Zoo'
#)

#data = [trace1, trace2]
layout = go.Layout(barmode='group')
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='grouped-bar')

Skip this value, is old style
Skip this value, is old style
Skip this value, is old style
Skip this value, is old style
FlashLFQ (PMID:29083185) Label Free Analysis MS1
MaxQuant Label Free Analysis MS1
Skyline Label Free Analysis MS1
Skyline Other Quant analysis
MaxQuant TMT analysis
MaxQuant SILAC analysis
MaxQuant Label Free Analysis MS1
MaxQuant Phospho Quant analysis
MaxQuant TMT analysis
MaxQuant Label Free Analysis MS1
MaxQuant Phospho Quant analysis
MaxQuant Other Quant analysis
MaxQuant TMT analysis
MaxQuant SILAC analysis
MaxQuant Label Free Analysis MS1
MaxQuant Label Free Spectral Count Analysis
Skyline Label Free Analysis MS1
Skyline SWATH / DIA Quant
MaxQuant TMT analysis
MaxQuant SILAC analysis
MaxQuant Label Free Analysis MS1
PeptideAtlas (StPeter, Libra, XPress) TMT analysis
Skyline Label Free Analysis MS1
MaxQuant TMT analysis
MaxQuant SILAC analysis
MaxQuant Phospho Quant analysis
MaxQuant Other Quant analysis
MaxQuant Label Free Spectral Count Analysis
MaxQuant Label

ValueError: 
    Invalid value of type 'builtins.dict_keys' received for the 'x' property of bar
        Received value: dict_keys(['Philosopher (Alexey Lab)'])

    The 'x' property is an array that may be specified as a tuple,
    list, numpy array, or pandas Series